# Prueba de Concepto: OPC UA (Async) + SQLite

Este notebook se conecta al PLC mediante `asyncua`, lee tres variables críticas y las inserta en tiempo real en una base de datos local SQLite.

In [ ]:
import asyncio
import sqlite3
import logging
from datetime import datetime
from asyncua import Client, ua

# Configuraciones OPC UA
ENDPOINT = "opc.tcp://192.168.0.20:4840"
DB_PATH = "../database/datos.db"  # La base de datos estará en la carpeta 'database' de la raíz

# Nombres de las variables que queremos leer (copiadas del script anterior)
DB_NAME = "MES"
VAR_HEARTBEAT = "Heartbeat"
VAR_STATE = "MachineState"
VAR_TARGETSPD = "TargetSpeed"

logging.basicConfig(level=logging.INFO)
_logger = logging.getLogger('opcua_sqlite')

### 1. Preparar la Base de Datos SQLite
Creamos la tabla si no existe.

In [ ]:
def init_db():
    import os
    # Asegurarnos de que el directorio existe
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mes_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
            heartbeat INTEGER,
            machine_state INTEGER,
            target_speed REAL
        )
    ''')
    conn.commit()
    return conn

# Inicializamos la BD y dejamos la conexión abierta para las inserciones
conn = init_db()
_logger.info(f"Base de datos conectada en {DB_PATH}")

### 2. Búsqueda de Nodos (Helper)
Función para buscar los IDs de nuestros nodos por su nombre.

In [ ]:
async def find_node_by_name(client, db_name, var_name, max_depth=5):
    objects = client.nodes.objects
    
    async def walk(node, depth):
        if depth > max_depth: return None
        try: children = await node.get_children()
        except: return None
        
        for child in children:
            try: bname = (await child.read_browse_name()).Name
            except: continue
            
            if bname == db_name:
                # Buscar dentro del DB
                db_children = await child.get_children()
                for var in db_children:
                    if (await var.read_browse_name()).Name == var_name:
                        return var
            
            found = await walk(child, depth + 1)
            if found: return found
        return None

    return await walk(objects, 0)

### 3. Ciclo Principal de Lectura y Guardado
Este bucle se ejecutará hasta que detengas la celda.

In [ ]:
async def main_loop():
    _logger.info(f"Conectando a {ENDPOINT}...")
    async with Client(url=ENDPOINT) as client:
        _logger.info("Conectado a OPC UA.")
        
        # 1. Resolver los nodos una única vez
        _logger.info("Buscando los nodos en el servidor...")
        node_hb = await find_node_by_name(client, DB_NAME, VAR_HEARTBEAT)
        node_st = await find_node_by_name(client, DB_NAME, VAR_STATE)
        node_sp = await find_node_by_name(client, DB_NAME, VAR_TARGETSPD)
        
        if not node_hb or not node_st or not node_sp:
            _logger.error("❌ No se han encontrado todos los nodos. Revisa los nombres.")
            return
            
        _logger.info(f"Nodos encontrados.\n - Heartbeat: {node_hb}\n - Status: {node_st}\n - Speed: {node_sp}")
        _logger.info("--- INICIANDO LECTURA (Pulsa el botón Stop de la celda para detener) ---")
        
        cursor = conn.cursor()
        
        # 2. Bucle infinito de lectura e inserción
        try:
            while True:
                # Leer valores de OPC UA
                hb = await node_hb.read_value()
                st = await node_st.read_value()
                sp = await node_sp.read_value()
                
                # Insertar en SQLite
                timestamp = datetime.now().isoformat()
                cursor.execute(
                    "INSERT INTO mes_data (timestamp, heartbeat, machine_state, target_speed) VALUES (?, ?, ?, ?)",
                    (timestamp, hb, st, sp)
                )
                conn.commit()
                
                print(f"[{timestamp}] Insertado -> Heartbeat: {hb} | State: {st} | Speed: {sp}")
                
                await asyncio.sleep(1.0) # Esperar 1 segundo antes de la siguiente lectura
                
        except asyncio.CancelledError:
            _logger.info("Lectura detenida por el usuario.")

# Ejecutamos el bucle (Nota: En un Notebook se puede usar 'await' directamente en la celda)
try:
    task = asyncio.create_task(main_loop())
    await task
except KeyboardInterrupt:
    task.cancel()

### 4. Verificar los datos con Pandas
Una vez detenida la lectura anterior, puedes correr esta celda para comprobar que la tabla tiene registros guardados.

In [ ]:
import pandas as pd

# Leemos la tabla entera de SQLite y la ponemos en un DataFrame
df = pd.read_sql_query("SELECT * FROM mes_data ORDER BY id DESC LIMIT 10", conn)

print("Últimos 10 registros en SQLite:")
display(df)